Open the bash container:

In [ ]:
img_path=../0_resources/vsn_cache/vibsinglecellnf-singlecelltoolkit-2022-07-07-0638c1d.img
singularity exec --cleanenv -H $PWD -B /dodrio,/readonly/dodrio,/tmp $img_path bash

Generate fragments:

In [2]:
export POLARS_MAX_THREADS=8
export OMP_NUM_THREADS=8

subset=UCS
for file in ../full_1_vsn_preprocessing/full_preprocessing_out/data/reports/barcard/*$subset*tomerge.tsv
do
    echo $file
    sample=(`basename $file`)
    sample=${sample%.FULL.tomerge.tsv}.LIBDS
    # echo $sample
    jaccardfile=$file
    infile=../libds_1_vsn_preprocessing/libds_preprocessing_out/data/fragments/$sample.fragments.raw.tsv.gz
    outfile=fixedcells_preprocessing_out/data/fragments/$sample.fragments.tsv
    outfile2=$outfile.gz

    if [ -f "$outfile" ]; then
        echo "$outfile exists, skipping"
    elif [ -f "$outfile2" ]; then
        echo "$outfile2 exists, skipping"
    else 
        echo "$outfile does not exist, generating!"
        merge_cbs_over_jaccard_index_threshold.py -j $jaccardfile -i $infile -o $outfile &
    fi
done

full_preprocessing_out/data/fragments/BIO_ddseq_1.tomerge.tsv
full_preprocessing_out/data/fragments/BIO_ddseq_1.fragments.tsv does not exist, generating!
full_preprocessing_out/data/fragments/BIO_ddseq_2.tomerge.tsv
full_preprocessing_out/data/fragments/BIO_ddseq_2.fragments.tsv.gz exists, skipping
full_preprocessing_out/data/fragments/BIO_ddseq_3.tomerge.tsv
full_preprocessing_out/data/fragments/BIO_ddseq_3.fragments.tsv.gz exists, skipping
full_preprocessing_out/data/fragments/BIO_ddseq_4.tomerge.tsv
full_preprocessing_out/data/fragments/BIO_ddseq_4.fragments.tsv.gz exists, skipping
full_preprocessing_out/data/fragments/BRO_mtscatac_1.tomerge.tsv
full_preprocessing_out/data/fragments/BRO_mtscatac_1.fragments.tsv.gz exists, skipping
full_preprocessing_out/data/fragments/BRO_mtscatac_2.tomerge.tsv
full_preprocessing_out/data/fragments/BRO_mtscatac_2.fragments.tsv.gz exists, skipping
full_preprocessing_out/data/fragments/CNA_10xmultiome_1.tomerge.tsv
full_preprocessing_out/data/fragment

Then bgzip the output:

In [5]:
module load HTSlib
subset=
for file in fixedcells_preprocessing_out/data/fragments/*$subset*.fragments.tsv
do
    echo $file
    if [ -f "$file.gz" ]; then
        echo exists
    else
        bgzip -@ 4 $file &
    fi
done

fixedcells_preprocessing_out/data/fragments/BIO_ddseq_1.FIXEDCELLS.fragments.tsv
exists
fixedcells_preprocessing_out/data/fragments/HAR_ddseq_1.FIXEDCELLS.fragments.tsv
exists
fixedcells_preprocessing_out/data/fragments/HAR_ddseq_2.FIXEDCELLS.fragments.tsv
exists
fixedcells_preprocessing_out/data/fragments/MDC_mtscatac_2.FIXEDCELLS.fragments.tsv
exists
fixedcells_preprocessing_out/data/fragments/SAN_10xmultiome_2.FIXEDCELLS.fragments.tsv
exists
fixedcells_preprocessing_out/data/fragments/TXG_10xv11_1.FIXEDCELLS.fragments.tsv
exists
fixedcells_preprocessing_out/data/fragments/TXG_10xv2_1.FIXEDCELLS.fragments.tsv
exists
fixedcells_preprocessing_out/data/fragments/TXG_10xv2_2.FIXEDCELLS.fragments.tsv
exists
[2]   Done                    bgzip -@ 4 $file


In [11]:
jobs

[1]   Done                    bgzip -@ 4 $file
[26]   Done                    bgzip -@ 4 $file
[35]   Done                    bgzip -@ 4 $file
[36]-  Done                    bgzip -@ 4 $file
[37]+  Done                    bgzip -@ 4 $file


and tabix index

In [14]:
module load HTSlib
subset=
for file in fixedcells_preprocessing_out/data/fragments/*$subset*.fragments.tsv.gz
do
    echo $file
    if [ -f "$file.gz.tbi" ]; then
        echo exists
    else
        tabix -p bed $file &
    fi
done

fixedcells_preprocessing_out/data/fragments/BIO_ddseq_1.FIXEDCELLS.fragments.tsv.gz
[1] 394316
fixedcells_preprocessing_out/data/fragments/BIO_ddseq_2.FIXEDCELLS.fragments.tsv.gz
[2] 394317
fixedcells_preprocessing_out/data/fragments/BIO_ddseq_3.FIXEDCELLS.fragments.tsv.gz
[3] 394318
fixedcells_preprocessing_out/data/fragments/BIO_ddseq_4.FIXEDCELLS.fragments.tsv.gz
[4] 394319
fixedcells_preprocessing_out/data/fragments/BRO_mtscatac_1.FIXEDCELLS.fragments.tsv.gz
[5] 394320
fixedcells_preprocessing_out/data/fragments/BRO_mtscatac_2.FIXEDCELLS.fragments.tsv.gz
[6] 394321
fixedcells_preprocessing_out/data/fragments/CNA_10xmultiome_1.FIXEDCELLS.fragments.tsv.gz
[7] 394322
fixedcells_preprocessing_out/data/fragments/CNA_10xmultiome_2.FIXEDCELLS.fragments.tsv.gz
[8] 394323
fixedcells_preprocessing_out/data/fragments/CNA_10xv11_1.FIXEDCELLS.fragments.tsv.gz
[9] 394324
fixedcells_preprocessing_out/data/fragments/CNA_10xv11_2.FIXEDCELLS.fragments.tsv.gz
[10] 394325
fixedcells_preprocessing_out/

In [7]:
prog=../0_resources/single_cell_toolkit_rust/target/release/add_new_bc_tag
prog=/dodrio/scratch/projects/starting_2022_023/benchmark/scatac_benchmark/0_resources/single_cell_toolkit_rust/add_new_bc_tag_if_old_found
for file in fixedcells_preprocessing_out/data/bam/UCS*possorted.bam
do
    echo $file
    sample=(`basename $file`)
    sample=${sample%_____R1.bwa.out.possorted.bam}.FIXEDCELLS
    echo $sample
    outfile=fixedcells_preprocessing_out/data/bam/$sample.possorted.dbfixed.bam
    bc_map=fixedcells_preprocessing_out/data/fragments/$sample.fragments.tsv.original_CB_to_merged_CBs.tsv
    ls $bc_map
    if [ -f "$outfile" ]; then
        echo "$outfile exists, skipping"
    else 
        echo "$outfile does not exist, generating!"
        $prog $file $outfile $bc_map CB DB &
    fi
done

full_preprocessing_out/data/bam/BIO_ddseq_1.FULL_____R1.bwa.out.possorted.bam
BIO_ddseq_1.FULL
full_preprocessing_out/data/fragments/BIO_ddseq_1.FULL.fragments.tsv.original_CB_to_merged_CBs.tsv
full_preprocessing_out/data/bam/BIO_ddseq_1.FULL.possorted.dbfixed.bam does not exist, generating!



In [2]:
for file in full_preprocessing_out/data/bam/UCS*.possorted.dbfixed.bam
do
    echo $file
    outfile=$file.bai
    if [ -f "$outfile" ]; then
        echo "$outfile exists, skipping"
    else 
        echo "$outfile does not exist, generating!"
        samtools index -@ 8 $file &
    fi
done

full_preprocessing_out/data/bam/BIO_ddseq_1.FULL.possorted.dbfixed.bam
full_preprocessing_out/data/bam/BIO_ddseq_1.FULL.possorted.dbfixed.bam.bai does not exist, generating!
full_preprocessing_out/data/bam/BIO_ddseq_2.FULL.possorted.dbfixed.bam
full_preprocessing_out/data/bam/BIO_ddseq_2.FULL.possorted.dbfixed.bam.bai exists, skipping
full_preprocessing_out/data/bam/BIO_ddseq_3.FULL.possorted.dbfixed.bam
full_preprocessing_out/data/bam/BIO_ddseq_3.FULL.possorted.dbfixed.bam.bai exists, skipping
full_preprocessing_out/data/bam/BIO_ddseq_4.FULL.possorted.dbfixed.bam
full_preprocessing_out/data/bam/BIO_ddseq_4.FULL.possorted.dbfixed.bam.bai exists, skipping
full_preprocessing_out/data/bam/BRO_mtscatac_1.FULL.possorted.dbfixed.bam
full_preprocessing_out/data/bam/BRO_mtscatac_1.FULL.possorted.dbfixed.bam.bai exists, skipping
full_preprocessing_out/data/bam/BRO_mtscatac_2.FULL.possorted.dbfixed.bam
full_preprocessing_out/data/bam/BRO_mtscatac_2.FULL.possorted.dbfixed.bam.bai exists, skipping